### 裁剪逻辑测试

1. 先解析目录，得到目录中所有标题和页面的信息

前20页作为目录的主要区域，在这20页内进行查找

2. 对目录中的标题和需要裁剪的关键字进行匹配

3. 裁剪

In [ ]:
import fitz  
import re
import difflib
import os


def compute_page_offset(doc, search_range=30):
    """
    计算 PDF 物理页码与逻辑页码的偏移量。
    策略：扫描前 N 页，寻找页面底部标有 "- 1 -" 或 "1" 的页面。
    该页面的物理索引 (index) 即为偏移量 offset。
    例如：第 5 页印着 "1"，说明 offset = 4 (因为 index 是 4)。
    """
    offset = 0
    try:
        # 匹配页面底部常见的页码格式： "1", "- 1 -", "Page 1"
        # 注意：很多文档第一页正文不标页码，所以我们找 "1" 或者 "2" 倒推
        page_num_patterns = [
            r"^\s*[-—]?\s*1\s*[-—]?\s*$",  # - 1 -
            r"^\s*1\s*$",                  # 1
            r"第\s*1\s*页"                 # 第 1 页
        ]
        
        # 扫描前 search_range 页
        for i in range(min(search_range, doc.page_count)):
            page = doc[i]
            # 获取页面文本，限制只看底部 10% 的区域（页脚通常在这里）
            rect = page.rect
            footer_rect = fitz.Rect(0, rect.height * 0.9, rect.width, rect.height)
            text = page.get_text("text", clip=footer_rect).strip()
            
            # 检查是否匹配“1”
            for pat in page_num_patterns:
                if re.search(pat, text):
                    print(f"🔍 在物理第 {i+1} 页底端发现页码 '1'，计算偏移量 offset = {i}")
                    return i
        
        # 备选策略：如果找不到 "1"，尝试找 "2" 或者是 "目录" 结束后的下一页
        # 这里为了稳健，如果找不到，尝试通过目录的第一条目倒推
        # 但目前保持 0 是最安全的默认值（即假设封面就是第1页）
        print("⚠️ 未能在页脚自动检测到起始页码 '1'，默认 offset = 0")
        return 0

    except Exception as e:
        print(f"计算偏移量出错: {e}")
        return 0


# ========================================================
# 1. 核心：解析目录生成字典
# ========================================================
def parse_toc_to_dict(doc, max_scan_pages=20):
    """
    解析PDF目录，返回结构化字典：
    {
        "clean_title_string": [start_page, end_page],
        ...
    }
    """
    toc_list = [] # 临时存储 [(title, page), ...]
    full_toc_text = ""

    # --- A. 提取前N页文本 ---
    for i in range(min(max_scan_pages, doc.page_count)):
        try:
            page_text = doc[i].get_text()
            if page_text:
                full_toc_text += page_text + "\n"
        except:
            continue

    # --- B. 清洗文本 (关键步骤) ---
    # 1. 去除目录中的虚线/点 (如 "......")
    clean_text = re.sub(r"[…\.．]{2,}", " ", full_toc_text)
    # 2. 尝试修复换行 (有些标题被断成两行，通常下一行是页码)
    # 这一步比较激进，根据实际情况微调
    # clean_text = re.sub(r'\n\s*(\d+)', r' \1', clean_text) 

    # --- C. 正则匹配 (提取 标题 + 页码) ---
    # 匹配模式：行首(可能含章节号) + 内容 + 空格 + 页码(行尾)
    # (?m) 开启多行模式
    # ([^\n\d]+?) 匹配非数字的标题部分 (非贪婪)
    # (\d+) 匹配页码
    pattern = r"(?m)^\s*(.*?)\s+(\d+)\s*$"
    matches = re.findall(pattern, clean_text)

    for title, page_str in matches:
        # 清洗标题：去掉首尾空格、去掉末尾的点
        clean_title = title.strip().rstrip('.').rstrip()
        # 去掉中间的所有空白字符（方便后续模糊匹配）
        compact_title = re.sub(r"\s+", "", clean_title)
        
        # 过滤掉过短的误判 (比如只有 "1")
        if len(compact_title) > 1:
            try:
                page_num = int(page_str)
                # 过滤掉页码大得离谱的误判
                if page_num <= doc.page_count + 10: 
                    toc_list.append((compact_title, page_num))
            except:
                continue

    # --- D. 构建闭环字典 {Title: [Start, End]} ---
    toc_dict = {}
    total_items = len(toc_list)
    
    if total_items == 0:
        return {}

    for i in range(total_items):
        title, start_p = toc_list[i]
        
        # 确定结束页：默认为下一条目的开始页
        if i < total_items - 1:
            next_start_p = toc_list[i+1][1]
            # 逻辑修正：有时候下一章可能和当前章在同一页，或者页码回溯（目录页码错误）
            # 我们取 max(start_p, next_start_p) 保证不倒退
            end_p = max(start_p, next_start_p) 
        else:
            # 最后一项，结束页为文档总页数
            end_p = doc.page_count

        # 存入字典
        # 注意：如果有重名标题（极少见），后面会覆盖前面，或者可以存成列表
        toc_dict[title] = [start_p, end_p]
    # print("current file toc_dict:")
    # print(toc_dict)

    return toc_dict

# ========================================================
# 2. 匹配逻辑：在字典中查表
# ========================================================
def match_section_from_dict(toc_dict, keyword, threshold=0.4):
    """
    在目录字典中寻找最匹配 keyword 的条目
    返回: (start_page, end_page, matched_title)
    """
    if not toc_dict:
        return None, None, None

    best_score = 0
    best_key = None

    # 遍历所有标题进行打分
    for title in toc_dict.keys():
        # 1. 字符覆盖率 (解决 "存在问题" vs "存在的主要问题")
        keyword_chars = set(keyword)
        title_chars = set(title)
        common_chars = keyword_chars.intersection(title_chars)
        coverage = len(common_chars) / len(keyword_chars) if keyword_chars else 0
        
        # 2. 序列相似度 (difflib)
        seq_score = difflib.SequenceMatcher(None, keyword, title).ratio()
        
        # 3. 综合得分
        # 如果关键词包含在标题里，给予极高权重
        if keyword in title:
            final_score = 1.0
        else:
            final_score = max(coverage, seq_score)

        if final_score > best_score:
            best_score = final_score
            best_key = title

    print(f"🔍 搜索关键词: '{keyword}' | 最佳匹配: '{best_key}' (得分: {best_score:.2f})")

    if best_key and best_score >= threshold:
        pages = toc_dict[best_key]
        return pages[0], pages[1], best_key
    else:
        return None, None, None

# ========================================================
# 3. 主入口：裁剪函数
# ========================================================
def extract_section_to_pdf(pdf_path, output_path, section_keyword="问题"):
    src_doc = None
    out_doc = None
    try:
        src_doc = fitz.open(pdf_path)
        
        # 1. 计算偏移量 (Offset)
        # 逻辑页码 (目录上的 1) + Offset = 物理索引 (FitZ 的 4)
        offset = compute_page_offset(src_doc)
        print(f"📄 文档总页数: {src_doc.page_count}, 计算偏移量 Offset = {offset}")
        
        # 2. 解析目录
        print("正在解析目录结构...")
        toc_dict = parse_toc_to_dict(src_doc)
        
        if not toc_dict:
            print("⚠️ 文本目录解析失败，尝试使用书签...")
            # (这里省略了书签逻辑，如果需要可以加上)
            return False

        # 3. 匹配区间 (得到的是目录上的逻辑页码，例如 5 -> 8)
        start_logic, end_logic, matched_title = match_section_from_dict(toc_dict, section_keyword)
        
        if start_logic is None:
            print("❌ 未找到匹配章节")
            return False

        # 4. 坐标转换：逻辑页码 -> 物理索引
        # 公式：物理索引 = 逻辑页码 + offset - 1
        # (-1 是因为页码从1开始，索引从0开始)
        # 例如：目录写第1页，offset是4(即第5页是正文P1)，则索引 = 1 + 4 - 1 = 4 (正确)
        
        start_idx = start_logic + offset - 1
        
        # 处理最后一章的情况 (end_logic 为 99999)
        if end_logic > 90000:
            end_idx = src_doc.page_count - 1 # 直到文档末尾
        else:
            end_idx = end_logic + offset - 1

        # 5. 边界修正
        if start_idx < 0: start_idx = 0
        if start_idx >= src_doc.page_count: 
            print("❌ 计算出的起始页超出文档范围")
            return False
            
        if end_idx >= src_doc.page_count: end_idx = src_doc.page_count - 1
        
        # 关键修正：如果算出来的 end_idx 比 start_idx 还小（目录页码标错了），强制取一页
        if end_idx < start_idx: 
            # 尝试往后多取几页，比如默认提取 3 页
            print("⚠️ 结束页码异常，默认提取 3 页")
            end_idx = min(start_idx + 2, src_doc.page_count - 1)

        print(f"✅ 执行裁剪: {matched_title}")
        print(f"   逻辑页码: {start_logic} -> {end_logic}")
        print(f"   物理索引: {start_idx} -> {end_idx} (Offset={offset})")

        # 6. 保存
        out_doc = fitz.open()
        # insert_pdf 的 to_page 是包含在内的，所以不需要 -1
        out_doc.insert_pdf(src_doc, from_page=start_idx, to_page=end_idx)
        out_doc.save(output_path)
        return True

    except Exception as e:
        print(f"裁剪过程异常: {e}")
        import traceback
        traceback.print_exc()
        return False
    finally:
        if src_doc: src_doc.close()
        if out_doc: out_doc.close()

In [16]:
# 测试10个文件的裁剪
import os
path = '/mnt/e/共享文件夹/土地整治/非试点2'
files = [os.path.join(path,i) for i in os.listdir(path) if i.endswith('.pdf')][:10]

for file in files:
    print("file:", os.path.basename(file))
    save_path = '/mnt/c/Users/Administrator/Downloads/test'
    file_save_path = os.path.join(save_path, os.path.basename(file).replace('.pdf','_裁剪测试.pdf'))
    extract_section_to_pdf(file, file_save_path, section_keyword="存在问题")

file: 东莞市东坑镇全域土地综合整治实施方案.pdf
🔍 在物理第 13 页底端发现页码 '1'，计算偏移量 offset = 12
📄 文档总页数: 469, 计算偏移量 Offset = 12
正在解析目录结构...
current file toc_dict:
{'一、基本情况': [1, 1], '（一）区域概况': [1, 23], '（二）存在问题': [23, 27], '（三）整治区域及实施期限': [27, 32], '（四）编制依据': [32, 39], '（五）基础数据': [39, 40], '二、可行性分析及评估': [40, 40], '（一）已有工作基础分析': [40, 45], '（二）空间布局优化可行性分析': [45, 70], '（三）整治可行性分析': [70, 86], '（四）产业导入可行性分析': [86, 91], '（五）风险评估及应对': [91, 96], '三、整治目标': [96, 96], '（一）目标设置': [96, 107], '（二）生产要素创新性配置制度探索': [107, 112], '四、建设任务与子项目安排': [112, 112], '（一）规划落实及其他方案衔接': [112, 150], '（二）重点建设任务落实': [150, 154], '（三）子项目安排': [154, 201], '五、精品工程': [201, 201], '（一）岭南古韵•产业新城——东坑城乡融合示范圈': [201, 221], '（二）现代农业•生态文旅——东坑农文旅融合示范圈': [221, 232], '六、投资测算及资金筹措': [232, 232], '（一）投资测算': [232, 237], '（二）资金筹措': [237, 240], '（三）项目融资及债务清偿能力分析': [240, 240], '（四）资金平衡分析': [240, 258], '七、预期效益和项目特色': [258, 258], '（一）预期效益': [258, 263], '（二）项目特色': [263, 265], '八、实施保障': [265, 265], '（一）组织保障': [265, 265], '（二）资金保障': [265, 266], '（三）制度保障': [266, 267], '（四）监督保

In [21]:
a = ["parsed_raw_matrix.csv", "parsed_potential.csv"]

for i in a:
    if i.startswith('parsed') and not i.endswith("matrix.csv"):
        print(i)

parsed_potential.csv


In [1]:
name = '东莞-厚街_LandUse.pdf'
from utils.utils_pdf import extract_info
info = extract_info(name)
print(info)

{'原始文件名': '东莞-厚街_LandUse.pdf', '文件名': '东莞-厚街_LandUse', '城市': '东莞', '地区/县': '厚街_LandUse', '详细单元': '无'}


In [ ]:
origion = ['存在_产业发展与用地供给矛盾', '存在_人与自然的矛盾', '存在_人地协调矛盾',
       '存在_低效用地问题', '存在_耕地碎片化']
new = ['低效用地问题_排序', '人地协调矛盾_排序', '产业发展与用地供给矛盾_排序',
       '人与自然的矛盾_排序', '耕地碎片化_排序']
map = []
for issue in origion:
    if issue.startswith('存在_'):
        key = issue.replace('存在_','')
        for n in new:
            if n.startswith(key):
                map.append((issue, n))
                break

[('存在_产业发展与用地供给矛盾', '产业发展与用地供给矛盾_排序'), ('存在_人与自然的矛盾', '人与自然的矛盾_排序'), ('存在_人地协调矛盾', '人地协调矛盾_排序'), ('存在_低效用地问题', '低效用地问题_排序'), ('存在_耕地碎片化', '耕地碎片化_排序')]
